The first 6 optimisation attempts were focused around feature selection based on the importance and correlation. 

The optimisation 7, we compared the training and testing accuracy and Optimisation 8 was performed to tune the number of trees (n_estimator). 

In Optimisation 9 and 10, we tried with the larger model to learn the best range of hyperparameters (e.g. alpha, lamba etc) and in the final attempt we are trying to use the shorter version of the model by trying to incorporate the hyperparameter outputs that we obtained from Optimisation 8 and 10.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/encoded-df3/encoded_df3.csv


In [2]:
# Import dependencies 
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

plt.style.use('dark_background')

In [3]:
# Import encoded datafram
encoded_df3 = pd.read_csv("../input/encoded-df3/encoded_df3.csv")
encoded_df3

,state_beer,style,availability,abv,types,taste
0,36,German Hefeweizen,2,5.0,4,4.50
1,36,German Hefeweizen,6,5.6,2,4.50
2,43,German Hefeweizen,6,5.4,1,4.00
3,37,German Hefeweizen,4,5.0,10,4.25
4,46,German Hefeweizen,2,4.5,3,4.00
...,...,...,...,...,...,...
443995,4,Stout,6,5.8,3,4.50
443996,10,Stout,2,8.0,2,3.50
443997,4,Stout,2,11.0,10,5.00
443998,22,Stout,2,13.1,5,4.75


In [4]:
# import dependencies
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [5]:
#encode string values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_df4 = encoded_df3.copy()
encoded_df4['style'] = le.fit_transform(encoded_df4['style'])
encoded_df4

,state_beer,style,availability,abv,types,taste
0,36,18,2,5.0,4,4.50
1,36,18,6,5.6,2,4.50
2,43,18,6,5.4,1,4.00
3,37,18,4,5.0,10,4.25
4,46,18,2,4.5,3,4.00
...,...,...,...,...,...,...
443995,4,36,6,5.8,3,4.50
443996,10,36,2,8.0,2,3.50
443997,4,36,2,11.0,10,5.00
443998,22,36,2,13.1,5,4.75


In [6]:
# Define the target set
y = encoded_df4["style"].values
y

array([18, 18, 18, ..., 36, 36, 36])

In [7]:
#Drop y from the main df
encoded_df4 = encoded_df4.drop(['style'], axis=1)
encoded_df4

,state_beer,availability,abv,types,taste
0,36,2,5.0,4,4.50
1,36,6,5.6,2,4.50
2,43,6,5.4,1,4.00
3,37,4,5.0,10,4.25
4,46,2,4.5,3,4.00
...,...,...,...,...,...
443995,4,6,5.8,3,4.50
443996,10,2,8.0,2,3.50
443997,4,2,11.0,10,5.00
443998,22,2,13.1,5,4.75


In [8]:
#define the features set
X = encoded_df4.copy()
X

,state_beer,availability,abv,types,taste
0,36,2,5.0,4,4.50
1,36,6,5.6,2,4.50
2,43,6,5.4,1,4.00
3,37,4,5.0,10,4.25
4,46,2,4.5,3,4.00
...,...,...,...,...,...
443995,4,6,5.8,3,4.50
443996,10,2,8.0,2,3.50
443997,4,2,11.0,10,5.00
443998,22,2,13.1,5,4.75


In [9]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
best_trial1 = {'lambda': 0.6961487559906326, 'alpha': 0.010396164166391456, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.008, 'max_depth': 17, 'random_state': 2020, 'min_child_weight': 1, 'tree_method': 'gpu_hist','n_estimators': 200 }
best_trial1

{'lambda': 0.6961487559906326,
 'alpha': 0.010396164166391456,
 'colsample_bytree': 0.9,
 'subsample': 0.7,
 'learning_rate': 0.008,
 'max_depth': 17,
 'random_state': 2020,
 'min_child_weight': 1,
 'tree_method': 'gpu_hist',
 'n_estimators': 200}

In [12]:
# create an optimized xgb classifier
xgb_model1 = XGBClassifier(**best_trial1)
# fit the model
xgb_model1 = xgb_model1.fit(X_train_scaled, y_train)

In [13]:
# make predictions for test data
predictions1 = xgb_model1.predict(X_test_scaled)

In [14]:
#calculate matrix and accuracy score
cm1 = confusion_matrix(y_test, predictions1)
cm_df1 = pd.DataFrame(cm1)
accuracy = accuracy_score(y_test, predictions1)

In [15]:
# Display results

print("Confusion Matrix")
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Classification Report")
print(classification_report(y_test, predictions1))

Confusion Matrix
Accuracy: 76.30%
Classification Report
              precision    recall  f1-score   support

           0       0.74      0.77      0.75      3651
           1       0.77      0.82      0.80      3689
           2       0.80      0.84      0.82      3668
           3       0.84      0.89      0.86      3638
           4       0.75      0.86      0.80      3584
           5       0.76      0.69      0.72      3683
           6       0.71      0.83      0.76      3681
           7       0.69      0.67      0.68      3685
           8       0.72      0.65      0.68      3671
           9       0.71      0.62      0.66      3747
          10       0.77      0.81      0.79      3637
          11       0.85      0.90      0.87      3610
          12       0.59      0.55      0.57      3701
          13       0.83      0.88      0.86      3724
          14       0.77      0.66      0.71      3687
          15       0.82      0.85      0.83      3593
          16       0.87  

In [16]:
train_predictions = xgb_model1.predict(X_train_scaled)
print(classification_report(y_train, train_predictions))

              precision    recall  f1-score   support

           0       0.78      0.79      0.79      7449
           1       0.81      0.86      0.83      7411
           2       0.81      0.87      0.83      7432
           3       0.86      0.90      0.88      7462
           4       0.79      0.88      0.84      7516
           5       0.80      0.72      0.76      7417
           6       0.74      0.87      0.80      7419
           7       0.74      0.73      0.73      7415
           8       0.76      0.69      0.72      7429
           9       0.75      0.70      0.72      7353
          10       0.79      0.83      0.81      7463
          11       0.87      0.91      0.89      7490
          12       0.67      0.62      0.65      7399
          13       0.85      0.90      0.87      7376
          14       0.82      0.71      0.76      7413
          15       0.84      0.87      0.85      7507
          16       0.88      0.92      0.90      7459
          17       0.83    

In [17]:
#Export testing classification report
df1 = pd.DataFrame(classification_report(y_test, predictions1, output_dict=True))
df1.to_csv('test_classification_report_xgboost.csv',index=False)


In [18]:
#Export training classification report
df2 = pd.DataFrame(classification_report(y_train, train_predictions, output_dict=True))
df2.to_csv('train_classification_report_xgboost.csv',index=False)